在本章中，将学习：1、如何使用模块unittest中的工具来为函数和类编写测试；2、如何编写继承unittest.TestCase的类，以及如何编写测试方法，以核实函数和类的行为符合预期；3、如何使用方法setUp()来根据类高效地创建实例并设置其属性，以便在类的所有测试方法中都可使用它们。

11.1 测试函数

要学习测试，得有要测试的代码。下面是一个简单的函数，它接受名和姓并返回整洁的姓名,将其存为name_function.py：

In [1]:
# name_function.py
def get_formatted_name(first,last):
    """Generate a neatly formatted full name."""
    full_name = first+' '+last
    return full_name.title()

为核实get_formatted_name() 像期望的那样工作，我们来编写一个使用这个函数的程序。程序names.py让用户输入名和姓，并显示整洁的全名：

In [3]:
from name_function import get_formatted_name 

print("Enter 'q' at any time to quit.")
while True:
    first = input('\nPlease give me a first name:')
    if first == 'q':
        break
    last=input('\nPlease give me a last name:')
    if last == 'q':
        break
        
    formatted_name = get_formatted_name(first,last)
    print("Neatly formatted name:"+formatted_name+'.')

Enter 'q' at any time to quit.

Please give me a first name:lily

Please give me a last name:collins
Neatly formatted name:Lily Collins.

Please give me a first name:q


从上述输出可知，合并得到的姓名正确无误。现在假设我们要修改get_formatted_name() ，使其还能够处理中间名。为此我们需要修改函数,并运行name.py，输入一些名字进行测试。这样略显繁琐，python提供了另一种高效的测试方式。

11.1.1 单元测试和测试用例

Python标准库中的模块unittest 提供了代码测试工具。1、单元测试 用于核实函数的某个方面没有问题；2、测试用例 是一组单元测试，这些单元测试一起核实函数在各种情形下的行为都符合要求。3、全覆盖式测试用例包含一整套单元测试，涵盖了各种可能的函数使用方式。

11.1.2 可通过的测试

In [2]:
# test_name_function.py
import unittest  
from name_function import get_formatted_name

class NamesTestCase(unittest.TestCase):
    """测试name_function.py"""
    def test_first_last_name(self):
        """能够正确地处理像Janis Joplin这样的姓名吗？"""
        formatted_name=get_formatted_name('janis','joplin')
        self.assertEqual(formatted_name,'Janis Joplin')
        
unittest.main(argv=['first-arg-is-ignored'],exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.068s

OK


创建了一个NamesTestCase的类，用于包含一系列针get_formatted_name()的单元测试。你可随便给这个类命名，但最好让它看起来与要测试的函数相关，并包含字样Test。这个类必须继承unittest.TestCase 类，这样Python才知道如何运行你编写的测试。

11.1.3 不能通过的的测试

修改get_formatted_name()的参数个数，再次用test_first_last_name测试，则结果会出错(缺少了一个位置实参)：

In [3]:
# name_function.py
def get_formatted_name(first,middle,last):
    """生成整洁的姓名"""
    full_name=first+' '+middle+' '+last
    return full_name.title()

In [4]:
import unittest
from name_function import get_formatted_name 
class NamesTestCase(unittest.TestCase):
    """测试name_function.py"""
    def test_first_last_name(self):
        """能够正确地处理像Janis Joplin这样的姓名吗？"""
        formatted_name = get_formatted_name('janis','joplin') 
        self.assertEqual(formatted_name,'Janis Joplin')
        
unittest.main(argv=['first-arg-is-ignored'], exit=False)

E
ERROR: test_first_last_name (__main__.NamesTestCase)
能够正确地处理像Janis Joplin这样的姓名吗？
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-4-0e9d9fa52786>", line 7, in test_first_last_name
    formatted_name = get_formatted_name('janis','joplin')
TypeError: get_formatted_name() missing 1 required positional argument: 'last'

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


11.1.4 测试未通过时怎么办

如果检查条件没错，测试通过表示函数内行为是对的，而测试未通过意味代码有问题。因此测试未通过时，不应修改测试，而应
找出函数代码中的错误。

在上一个示例中，get_formatted_name() 以前只需要两个实参，后面修改为三个实参(中间名加入）；如何使只有名和姓的通过测试呢？最佳办法是设置中间名可选（middle移动到形参末尾，默认为空字符）。

In [5]:
# name_function.py
def get_formatted_name(first, last, middle=''):
    """生成整洁的姓名"""
    if middle:
        full_name = first + ' ' + middle + ' ' + last
    else:
        full_name = first + ' ' + last
    return full_name.title()

In [6]:
import unittest
from name_function import get_formatted_name 
class NamesTestCase(unittest.TestCase):
    """测试name_function.py"""
    def test_first_last_name(self):
        """能够正确地处理像Janis Joplin这样的姓名吗？"""
        formatted_name = get_formatted_name('janis','joplin') 
        self.assertEqual(formatted_name,'Janis Joplin')
        
unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


11.1.5 添加新测试

再编写一个测试，用于测试包含中间名的姓名。为此，我们在NamesTestCase 类中再添加一个方法：

In [6]:
import unittest
from name_function import get_formatted_name
class NamesTestCase(unittest.TestCase):
    """测试name_function.py """
    def test_first_last_name(self):
        """能够正确地处理像Janis Joplin这样的姓名吗？"""
        formatted_name = get_formatted_name('janis','joplin')
        self.assertEqual(formatted_name,'Janis Joplin')
        
    def test_first_middle_last_name(self):
        """能够正确地处理像Wolfgang Amadeus Mozart这样的姓名吗？"""
        formatted_name = get_formatted_name('wolfgang', 'mozart', 'amadeus')
        self.assertEqual(formatted_name,'Wolfgang Amadeus Mozart')
        
unittest.main(argv=['first-arg-is-ignored'],exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


一个必须注意的点，所有的测试方法名必须以test_打头，这样它才会在我们运行test_name_function.py时自动运行。

### 11.1 作业

11-1 城市和国家 ：编写一个函数，它接受两个形参：一个城市名和一个国家名。这个函数返回一个格式为City, Country 的字符串，如Santiago, Chile 。将这个函数存储在一个名为city_functions.py的模块中。

In [7]:
# city_functions.py
def get_city_country(city,country):
    """存储城市名和国家名，返回City,Country"""
    city_info=city+','+country
    return city_info.title()

创建一个名为test_cities.py的程序，对刚编写的函数进行测试（别忘了，你需要导入模块unittest 以及要测试的函数）。编写一个名为test_city_country() 的
方法，核实使用类似于'santiago' 和'chile' 这样的值来调用前述函数时，得到的字符串是正确的。运行test_cities.py ，确认测
试test_city_country() 通过了。

In [8]:
# test_cities.py
import unittest
from city_functions import get_city_country
class CityTestCase(unittest.TestCase):
    """测试get_city_country.py"""
    def test_city_country(self):
        city_info= get_city_country('santiago','chile')
        self.assertEqual(city_info,'Santiago,Chile')
        
unittest.main(argv=['first-arg-is-ignored'],exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK


11-2 人口数量 ：修改前面的函数，使其包含第三个必不可少的形参population ，并返回一个格式为City, Country - population xxx 的字符串，
如Santiago, Chile - population 5000000 。运行test_cities.py，确认测试test_city_country() 未通过。

In [9]:
# city_functions.py
def get_city_country(city,country,population):
    """存储城市名和国家名，返回City,Country"""
    city_info=(city+','+country).title+'-population '+population
    return city_info

In [10]:
# test_cities.py
import unittest
from city_functions import get_city_country
class CityTestCase(unittest.TestCase):
    """测试get_city_country.py"""
    def test_city_country(self):
        city_info = get_city_country('santiago','chile')
        self.assertEqual(city_info,'Santiago,Chile')
        
unittest.main(argv=['first-arg-is-ignored'],exit=False)

E.
ERROR: test_city_country (__main__.CityTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-10-b0ff2a0456dc>", line 7, in test_city_country
    city_info = get_city_country('santiago','chile')
TypeError: get_city_country() missing 1 required positional argument: 'population'

----------------------------------------------------------------------
Ran 2 tests in 0.005s

FAILED (errors=1)


修改上述函数，将形参population 设置为可选的。再次运行test_cities.py，确认测试test_city_country() 又通过了。

In [11]:
# city_functions.py
def get_city_country(city,country,population=''):
    """存储城市名和国家名，返回City,Country"""
    if population:
        city_info=(city+','+country).title()+'-population '+population
    else:
        city_info=(city+','+country).title()
    return city_info

In [12]:
# test_cities.py
import unittest
from city_functions import get_city_country
class CityTestCase(unittest.TestCase):
    """测试get_city_country.py"""
    def test_city_country(self):
        city_info = get_city_country('santiago','chile')
        self.assertEqual(city_info,'Santiago,Chile')
        
unittest.main(argv=['first-arg-is-ignored'],exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK


再编写一个名为test_city_country_population() 的测试，核实可以使用类似于'santiago' 、'chile' 和'population=5000000' 这样的值来调用
这个函数。再次运行test_cities.py，确认测试test_city_country_population() 通过了。

In [13]:
# test_cities.py
import unittest
from city_functions import get_city_country
class CityTestCase(unittest.TestCase):
    """测试get_city_country.py"""
    def test_city_country(self):
        city_info = get_city_country('santiago','chile')
        self.assertEqual(city_info,'Santiago,Chile')
        
    def test_city_country_population(self):
        city_info = get_city_country('santiago','chile','5000000')
        self.assertEqual(city_info,'Santiago,Chile-population 5000000')    
        
unittest.main(argv=['first-arg-is-ignored'],exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK


## 11.2 测试类

在前一节中编写了针对单个函数的测试，下面来编写针对类的测试：

11.2.1 各种断言方法

Python在unittest.TestCase类中提供多种断言方法，断言方法测试该满足的条件是否满足。常用的6个断言方法如下：assertEqual(a,b);assertNotEqual(a,b);
assertTrue(x);assertFalse(x);assertIn(item,list);assertNotIn(item,list).

11.2.2 一个要测试的类

In [29]:
# 创建survey.py
class AnonymousSurvey():
    """收集匿名调查问卷的答案"""
    
    def __init__(self,question):
        """存储一个问题，并为存储答案做准备"""
        self.question = question
        self.responses =[]
        
    def show_question(self):
        """显示调查问卷"""
        print(question)
        
    def store_response(self,new_response):
        """存储单份调查问卷"""
        self.responses.append(new_response)
        
    def show_results(self):
        """显示收集到的所有问卷"""
        print('Survey results:')
        for response in self.responses:
            print('- '+response)

In [30]:
# languages_survey.py
from survey import AnonymousSurvey

#定义一个问题，并创建一个表示调查的AnonymousSurvey对象
question = "What language did you first learn to speak?"
my_survey = AnonymousSurvey(question)

#显示问题并存储答案
my_survey.show_question()
print("Enter 'q' at any time to quit.\n")
while True:
    response = input("Language: ")
    if response == 'q':
        break
    my_survey.store_response(response)
    
# 显示调查结果
print("\nThank you to everyone who participated in the survey!")
my_survey.show_results()

What language did you first learn to speak?
Enter 'q' at any time to quit.

Language: Chinese
Language: English
Language: Spanish
Language: q

Thank you to everyone who participated in the survey!
Survey results:
- Chinese
- English
- Spanish


11.2.3 测试AnonymousSurvey类

测试验证：如果用户面对调查问题时只提供了一个答案，这个答案也能被妥善地存储。使用方法assertIn() 来核实它包含在答案列表中：

In [32]:
# test_survey.py
import unittest 
from survey import AnonymousSurvey 
class TestAnonmyousSurvey(unittest.TestCase):
    """针对AnonymousSurvey类的测试"""
    def test_store_single_response(self):
        """测试单个答案会被妥善地存储"""
        question = "What language did you first learn to speak?" 
        my_survey = AnonymousSurvey(question)
        my_survey.store_response('English')
       
        self.assertIn('English', my_survey.responses)
        
unittest.main(argv=['first-arg-is-ignored'],exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.008s

OK


但只能收集一个答案的调查用途不大。下面来核实用户提供三个答案时，它们也将被妥善地存储。为此，再添加一个方法：

In [36]:
import unittest
from survey import AnonymousSurvey
class TestAnonymousSurvey(unittest.TestCase):
    """针对AnonymousSurvey类的测试"""
    def test_store_single_response(self):
        """测试单个答案会被妥善地存储"""
        question = "What language did you first learn to speak?" 
        my_survey = AnonymousSurvey(question)
        my_survey.store_response('English')
        self.assertIn('English', my_survey.responses)
        
    def test_store_three_responses(self):
        """测试三个答案会被妥善地存储"""
        question = "What language did you first learn to speak?"
        my_survey = AnonymousSurvey(question) 
        responses = ['English', 'Spanish', 'Mandarin']
        for response in responses:
            my_survey.store_response(response)
        for response in responses:
            self.assertIn(response, my_survey.responses)
            
unittest.main(argv=['first-arg-is-ignored'],exit=False)

.......
----------------------------------------------------------------------
Ran 7 tests in 0.007s

OK


11.2.4 方法setUp()

在前面的每个测试方法中，我们都需要创建了一个AnonymousSurvey实例，并每个方法中都创建了答案。使用unittest.TestCase类中的setUp()方法可以只创建一次，提高测试效率。

In [37]:
import unittest
from survey import AnonymousSurvey
class TestAnonymousSurvey(unittest.TestCase):
    """针对AnonymousSurvey类的测试"""
    def setUp(self):
        """创建一个调查对象和一组答案，供使用的测试方法使用"""
        question = "What language did you first learn to speak?" 
        self.my_survey = AnonymousSurvey(question) 
        self.responses = ['English', 'Spanish', 'Mandarin']
        
    def test_store_single_response(self):
        """测试单个答案会被妥善地存储"""
        self.my_survey.store_response(self.responses[0])
        self.assertIn(self.responses[0], self.my_survey.responses)
            
    def test_store_three_responses(self):
        """测试三个答案会被妥善地存储"""
        for response in self.responses:
            self.my_survey.store_response(response)
        for response in self.responses:
            self.assertIn(response, self.my_survey.responses)
                
unittest.main(argv=['first-arg-is-ignored'],exit=False)

.......
----------------------------------------------------------------------
Ran 7 tests in 0.008s

OK


测试自己编写的类时，方法setUp() 让测试方法编写起来更容易：可在setUp() 方法中创建一系列实例并设置它们的属性，再在测试方法中直接使用这些实例。相比于在每个
测试方法中都创建实例并设置其属性，这要容易得多。

### 11.2 作业

11-3 雇员 ：编写一个名为Employee 的类，其方法__init__() 接受名、姓和年薪，并将它们都存储在属性中。编写一个名为give_raise() 的方法，它默认将年薪增加5000美元，但也能够接受其他的年薪增加量。

In [40]:
# employee.py
class Employee():
    """存储雇员的名、姓、年薪"""
    def __init__(self,first_name,last_name,salary):
        self.firstname = first_name
        self.lastname = last_name
        self.salary = salary
        
    def give_raise(self,increase=''):
        if increase:
            #self.increasement=increase
            new_salary = int(self.salary)+int(increase)
        else:
            new_salary = self.salary + 5000
        return new_salary

为Employee 编写一个测试用例，其中包含两个测试方法：test_give_default_raise() 和test_give_custom_raise() 。使用方法setUp() ，以免在
每个测试方法中都创建新的雇员实例。运行这个测试用例，确认两个测试都通过了。

In [41]:
# test_employee.py
import unittest
#from employee import Employee

class TestEmployee(unittest.TestCase):
    def setUp(self):
        """创建一个雇员实例"""
        self.employee_example = Employee('Lily','Collins',100000)
    
    def test_give_default_raise(self):
        """测试默认年薪增长"""
        new_salary = self.employee_example.give_raise()
        self.assertEqual(new_salary,105000)
        
    def test_give_custom_raise(self):
        """测试其他年薪增长"""
        new_salary = self.employee_example.give_raise(12345)
        self.assertEqual(new_salary,112345)
        
unittest.main(argv=['first-arg-is-ignored'],exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.008s

OK
